# Data preprocessing

In [257]:
import re
import string
import json
import pandas as pd
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
import bugzilla
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Vladimir.Fedorov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Vladimir.Fedorov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [258]:
eclipse_data = "data/datasets2017/eclipse.csv"
data = pd.read_csv(eclipse_data)
data

,id,product,component,reporter,bug_status,resolution,priority,bug_severity,version,short_desc,opendate,dup_list,root_id,disc_id
0,1,Platform,Team,andre_weinand,CLOS,FIXE,P3,nor,2.0,Usability issue with external editors (1GE6IRL),2001-10-10,[183],1.0,1.0
1,2,Platform,Team,andre_weinand,RESO,FIXE,P5,nor,2.0,Opening repository resources doesn't honor typ...,2001-10-10,"[94, 9779, 15392]",2.0,2.0
2,3,Platform,Team,nick_edgar,RESO,FIXE,P5,nor,2.0,Sync does not indicate deletion (1GIEN83),2001-10-10,NaN,NaN,NaN
3,4,Platform,Team,grant_gayed,RESO,FIXE,P5,nor,2.0,need better error message if catching up over ...,2001-10-10,NaN,NaN,NaN
4,5,Platform,Team,john.arthorne,RESO,WONT,P3,nor,2.0,ISharingManager sharing API inconsistent (1GAU...,2001-10-10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503930,511876,JDT,Core,sebastian.millies,CLOS,DUPL,P3,nor,4.6,[1.8][inference] Error in type inference with ...,2017-02-07,NaN,508834.0,508834.0
503931,511877,Efxclips,Runtime,tom.schindl,NEW,---,P3,nor,2.4.0,Update Runtime Target to 4.6.2,2017-02-07,NaN,NaN,NaN
503932,511878,JDT,Core,sebastian.millies,NEW,---,P3,nor,4.6,[1.8][inference] Type inference error with lam...,2017-02-07,NaN,NaN,NaN
503933,511879,Virgo,document,fwaibel,RESO,FIXE,P3,nor,3.7.0,Migrate User Guide and Programmer Guide to Asc...,2017-02-07,NaN,NaN,NaN


## Removing invalid reports

In [259]:
#Removing the unuseful coloumns from the dataset
data = data.drop(columns=['product', 'component', 'reporter', 'priority', 'bug_severity',
                          'version', 'opendate', 'bug_status'])
#data = data[(data.bug_status == 'CLOS') | (data.bug_status == 'RESO') | (data.bug_status == 'VERI')]
data = data.dropna(axis=0, subset = ['root_id'])
data

,id,resolution,short_desc,dup_list,root_id,disc_id
0,1,FIXE,Usability issue with external editors (1GE6IRL),[183],1.0,1.0
1,2,FIXE,Opening repository resources doesn't honor typ...,"[94, 9779, 15392]",2.0,2.0
6,7,WONT,[Team API] move/copy semantics not preserved f...,[42962],7.0,7.0
9,10,FIXE,API - VCM event notification (1G8G6RR),[121067],10.0,10.0
19,20,FIXE,Workspace files,[40],20.0,20.0
...,...,...,...,...,...,...
503723,511669,DUPL,Add the functionality to load additional custo...,NaN,511670.0,511669.0
503724,511670,FIXE,Add the functionality to load additional custo...,[511669],511670.0,511669.0
503766,511712,---,Run cleanup action on eclipse.platform.ui/bund...,[511741],511712.0,511712.0
503795,511741,DUPL,Use enhanced for loop in eclipse.platform.ui p...,NaN,511712.0,511712.0


In [292]:
# TODO: try to mine all comments
URL = 'https://bugs.eclipse.org/bugs/xmlrpc.cgi'
api_key = 'bFHQVgL4SokpqlMoaKYigtjSnyyUgZRCsM7KJoBN'
bzapi = bugzilla.Bugzilla(url=URL, api_key=api_key)
ids = data['id'].tolist()
comments = bzapi.get_comments(ids[:1001])

In [291]:
import requests

headers = {"Content-type": "application/json"}
params = {
              "api_key": api_key,
          }

comments = []
for issue_id in ids: 
    URL = f"https://bugs.eclipse.org/bugs/rest/bug/{issue_id}/comment"
    resp = requests.get(URL , headers = headers, params = params)
    if resp.status_code != 200:
        print('error: ' + str(resp.status_code))
    else:
        comment = json.loads(resp.text)
        comments.append(comment)

ConnectionError: HTTPSConnectionPool(host='bugs.eclipse.org', port=443): Max retries exceeded with url: /bugs/rest/bug/1721/comment?api_key=bFHQVgL4SokpqlMoaKYigtjSnyyUgZRCsM7KJoBN (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f97068d1dc0>: Failed to establish a new connection: [Errno 65] No route to host'))

In [79]:
# comments['bugs']['472936']['comments'][0]['text']
comments = comments['bugs']
descrs = []
for issue_id in ids[:1001]:
    descrs.append(comments[str(issue_id)]['comments'][0]['text'])

In [211]:
data = data.iloc[:1001]
data['description'] = descrs
data.to_csv('csv/short_data_eclipse2017.csv', index=False)

<ipython-input-211-99f812485035>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['description'] = descrs


## Noise removal

In [212]:
data = pd.read_csv('csv/short_data_eclipse2017.csv')

In [178]:
def denoise_text(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*\f\w*', '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*]\)', '', text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    
    return text


'hello friend'

In [213]:
data['short_desc'] = data['short_desc'].apply(denoise_text)
data['description'] = data['description'].apply(denoise_text)
data

,id,resolution,short_desc,dup_list,root_id,disc_id,description
0,1,FIXE,usability issue with external editors,[183],1.0,1.0,setup a project that contains a gif resource ...
1,2,FIXE,opening repository resources doesnt honor type,"[94, 9779, 15392]",2.0,2.0,opening repository resource always open the de...
2,7,WONT,team api movecopy semantics not preserved for ...,[42962],7.0,7.0,the platform is able to notify people that a r...
3,10,FIXE,api vcm event notification,[121067],10.0,10.0,there seems to be a need for a vcm event notif...
4,20,FIXE,workspace files,[40],20.0,20.0,thought it would be useful if the set of repo ...
...,...,...,...,...,...,...,...
996,6454,FIXE,have option for showing packages hierarchically,"[10029, 24425]",6454.0,6454.0,im fully aware that java packages arent hierar...
997,6455,DUPL,npe kill javaindexer thread,NaN,15262.0,6455.0,i press ctrl shift t the progress informat...
998,6457,FIXE,rename project changes project location,[8445],6457.0,6457.0,create a project with an nondefault location ...
999,6460,DUPL,find dialog box obscures result,NaN,5969.0,2422.0,when using the findreplace dialog in the text ...


## Normalization

In [193]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def tokenize_and_normalize(text):
    result = []
    STOPWORDS = stopwords.words('english') + ['http', 'https']
    for token in simple_preprocess(text, min_len=3):
        if token not in STOPWORDS:
            result.append(lemmatize(token))
    if len(result) == 0:
        return np.nan
    return result

tokenize_and_normalize('hello world')

['hello', 'world']

In [214]:
data['short_desc'] = data['short_desc'].map(tokenize_and_normalize)
data['description'] = data['description'].map(tokenize_and_normalize)

In [215]:
data = data.dropna(axis=0, subset=['description'])
data.to_csv('preprocessed_short_data_eclipse2017.csv', index=False)
data

,id,resolution,short_desc,dup_list,root_id,disc_id,description
0,1,FIXE,"[usability, issue, external, editors]",[183],1.0,1.0,"[setup, project, contain, gif, resource, relea..."
1,2,FIXE,"[open, repository, resources, doesnt, honor, t...","[94, 9779, 15392]",2.0,2.0,"[open, repository, resource, always, open, def..."
2,7,WONT,"[team, api, movecopy, semantics, preserve, vcm...",[42962],7.0,7.0,"[platform, able, notify, people, resource, mov..."
3,10,FIXE,"[api, vcm, event, notification]",[121067],10.0,10.0,"[seem, need, vcm, event, notification, model, ..."
4,20,FIXE,"[workspace, file]",[40],20.0,20.0,"[think, would, useful, set, repo, connections,..."
...,...,...,...,...,...,...,...
996,6454,FIXE,"[option, show, package, hierarchically]","[10029, 24425]",6454.0,6454.0,"[fully, aware, java, package, arent, hierarchi..."
997,6455,DUPL,"[npe, kill, javaindexer, thread]",NaN,15262.0,6455.0,"[press, ctrl, shift, progress, information, wi..."
998,6457,FIXE,"[rename, project, change, project, location]",[8445],6457.0,6457.0,"[create, project, nondefault, location, ctmpmy..."
999,6460,DUPL,"[find, dialog, box, obscure, result]",NaN,5969.0,2422.0,"[use, findreplace, dialog, text, editor, match..."


## Seperation of Master Reports and Duplicate Reports

In [222]:
master_ids = np.unique(data['disc_id'].to_numpy())
master_reports = data[data.id.isin(master_ids)]
master_reports.to_csv('csv/short_master2017.csv', index=False)
master_reports

,id,resolution,short_desc,dup_list,root_id,disc_id,description
0,1,FIXE,"[usability, issue, external, editors]",[183],1.0,1.0,"[setup, project, contain, gif, resource, relea..."
1,2,FIXE,"[open, repository, resources, doesnt, honor, t...","[94, 9779, 15392]",2.0,2.0,"[open, repository, resource, always, open, def..."
2,7,WONT,"[team, api, movecopy, semantics, preserve, vcm...",[42962],7.0,7.0,"[platform, able, notify, people, resource, mov..."
3,10,FIXE,"[api, vcm, event, notification]",[121067],10.0,10.0,"[seem, need, vcm, event, notification, model, ..."
4,20,FIXE,"[workspace, file]",[40],20.0,20.0,"[think, would, useful, set, repo, connections,..."
...,...,...,...,...,...,...,...
993,6437,FIXE,"[timeout, launch]","[6919, 49659]",6437.0,6437.0,"[eclipse, corneri, continue, experience, probl..."
996,6454,FIXE,"[option, show, package, hierarchically]","[10029, 24425]",6454.0,6454.0,"[fully, aware, java, package, arent, hierarchi..."
997,6455,DUPL,"[npe, kill, javaindexer, thread]",NaN,15262.0,6455.0,"[press, ctrl, shift, progress, information, wi..."
998,6457,FIXE,"[rename, project, change, project, location]",[8445],6457.0,6457.0,"[create, project, nondefault, location, ctmpmy..."


In [220]:
dupl_reports = data[data.id.isin(master_ids) == False]
dupl_reports.to_csv('csv/short_dupl2017.csv', index=False)
dupl_reports

,id,resolution,short_desc,dup_list,root_id,disc_id,description
9,40,DUPL,"[need, connect, team, stream]",NaN,20.0,20.0,"[would, like, able, connect, team, stream, dir..."
10,48,DUPL,"[make, sure, future, store, project, reference...",NaN,22.0,22.0,"[project, reference, come, three, flavour, pro..."
12,61,DUPL,"[vcmmeta, show, change]",NaN,60.0,60.0,"[user, user, install, drop, declipseuser, java..."
16,94,DUPL,"[repositories, view, file, type, open, text, e...",NaN,2.0,2.0,"[browse, file, repositories, view, try, open, ..."
17,98,DUPL,[need],NaN,26.0,26.0,"[need, new, button, repo, connection, dialog, ..."
...,...,...,...,...,...,...,...
990,6425,DUPL,"[motif, dropdown, inconsistencies]",NaN,193318.0,5088.0,"[public, class, dropdowntest, public, static, ..."
991,6426,DUPL,"[automatic, version, number, contain, invalid,...",NaN,6149.0,6149.0,"[versioning, automatic, version, name, insert,..."
994,6449,DUPL,"[error, close, workspace]",NaN,5903.0,5674.0,"[close, workspace, get, message, something, li..."
995,6453,DUPL,"[npe, create, java, perspective]",NaN,6273.0,5925.0,"[upgrade, problems, lose, jre, settingsreset, ..."


## Documentation Preparation

In [249]:
# !pip install PyPDF2
from PyPDF2 import PdfFileReader
from pathlib import Path

from tika import parser

def get_text_from_pdf(file_name, start_page):
    #pdf = PdfFileReader(file_name)
    #text = ''
    raw = parser.from_file(file_name)
    return raw['content']
    #for i in range(start_page, len(pdf.pages)):
    #    text += pdf.pages[i].extractText()
    #return text

In [250]:
doc = get_text_from_pdf('docs/eclipse5.pdf', 32)

In [251]:
doc = denoise_text(doc)
doc = tokenize_and_normalize(doc)

In [256]:
with Path('docs/eclipse5.txt').open(mode='w') as f:
    f.write(str(doc))